    2018.4.25 Bookclub QuickNote
    
    p: distribution we want to know (model)
    q: distribution we use to approximate (guide)
    
    guide: Since we don't know how to describe model, we would get sample from guide (known distribution)

    ELBO: over q distribustion
    ELBO is lowerbound for p
    If you want to maximize log p , equals to maxmize ELBO
    
    KL divergence: Similarity between q and p
    In case that we want to get p, we intruduce a q (depending on our experince) 
        --> q and p get closer by KL divergence --> we can get good θ

    SVI: 
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO())
    Solve θ automatically
    
    A simple example:
    Slight bias coin, would like to come out how it tends to be positve (want to find q)
    Bernoulli distribution identity: p^x * (1-p) ^ 1-x 
     
    Prior: Use a assumed distrubution to guess p
    Posterior: Get experience from this time, and become next time's prior
    We can get a pre-assumption for p (value in 0~1), and that is close to 0.5
    --> Use beta distribution(10. 10)
    However, if we use Maximum likelihood, p is 0.6. (6 positive and 4 negative)
    
    Like GAN, Maximum likelihood and BETA distribution fight 
       
    程式：
    model: 
    guide: Beta(15, 15)
    
    (比較底層的內容，可能code沒有寫出來)
    取sample:
    如果要對一個f(x)取積分，但不知道長相
    那就是要做sample，去近似每段範圍的面積
    等距取樣可能會是不好的估計，該怎麼有效取樣呢？
    
    或是找一個function去蓋住f(x), 從function去sample, 用此function去推f(x)的長相
    
    在CV裡面，有甚麼example in reality?
    
    

# Overview
    Need 
        1. real data
        2. model: for guess distribution
        3. guide: introduce a parameterized distribution (感覺上是輔佐猜測model的distribution)
        4. optimizer

### variational inference (機率圖模型)

### Setup
    observations ⟺ pyro.observe
    latent random variables ⟺ pyro.sample
    parameters ⟺ pyro.param

### Model Learning

### Guide (variational distribution)
    Introduce a parameterized distribution
    1. model() and guide() needs to have a matching sample statement 
    2. Learning will be setup as an optimization problem where each iteration of training takes a step in  space that moves the guide closer to the exact posterior.

### ELBO (evidence lower bound)
     At high level variational inference is easy: all we need to do is define a guide and compute gradients of the ELBO

### SVI Class
    In Pyro the machinery for doing variational inference is encapsulated in the SVI class.

In [3]:
import pyro
from pyro.infer import SVI
# svi = SVI(model, guide, optimizer, loss="ELBO")

### Optimizers
    Pyro needs to dynamically generate an optimizer for each parameter the first time it appears during learning.

In [5]:
from pyro.optim import Adam

adam_params = {"lr": 0.005, "betas": (0.95, 0.999)}
optimizer = Adam(adam_params)

def per_param_callable(module_name, param_name, tags):
    if 'param_name' == 'my_special_parameter':
        return {"lr": 0.010}
    else:
        return {"lr": 0.001}

optimizer = Adam(per_param_callable)

### A simple example
#### We finish with a simple example. You’ve been given a two-sided coin. You want to determine whether the coin is fair or not, i.e. whether it falls heads or tails with the same frequency. You have a prior belief about the likely fairness of the coin based on two observations:

In [6]:
import pyro.distributions as dist

def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0 = Variable(torch.Tensor([10.0]))
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.observe("obs_{}".format(i), dist.bernoulli,
                     data[i], f)

    Our next task is to define a corresponding guide

In [7]:
def guide(data):
    # define the initial values of the two variational parameters
    log_alpha_q_0 = Variable(torch.Tensor([np.log(15.0)]),
                             requires_grad=True)
    log_beta_q_0 = Variable(torch.Tensor([np.log(15.0)]),
                            requires_grad=True)
    # register the two variational parameters with Pyro
    log_alpha_q = pyro.param("log_alpha_q", log_alpha_q_0)
    log_beta_q = pyro.param("log_beta_q", log_beta_q_0)
    alpha_q, beta_q = torch.exp(log_alpha_q), torch.exp(log_beta_q)
    # sample latent_fairness from the distribution
    # Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.beta, alpha_q, beta_q)

In [ ]:
# set up the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss="ELBO")

n_steps = 5000
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    
### Note that in the step() method we pass in the data, which then get passed to the model and guide.

    The only thing we’re missing at this point is some data. So let’s create some data and assemble all the code snippets above into a complete script:

In [10]:
from __future__ import print_function
import numpy as np
import torch
from torch.autograd import Variable
import pyro
from pyro.optim import Adam
from pyro.infer import SVI
import pyro.distributions as dist

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(6):
    data.append(Variable(torch.ones(1)))
for _ in range(4):
    data.append(Variable(torch.zeros(1)))

def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = Variable(torch.Tensor([10.0]))
    beta0 = Variable(torch.Tensor([10.0]))
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.beta, alpha0, beta0)
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.observe("obs_{}".format(i), dist.bernoulli, data[i], f)

def guide(data):
    # define the initial values of the two variational parameters
    # we initialize the guide near the model prior (except a bit sharper)
    log_alpha_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    log_beta_q_0 = Variable(torch.Tensor([np.log(15.0)]), requires_grad=True)
    # register the two variational parameters with Pyro
    log_alpha_q = pyro.param("log_alpha_q", log_alpha_q_0)
    log_beta_q = pyro.param("log_beta_q", log_beta_q_0)
    alpha_q, beta_q = torch.exp(log_alpha_q), torch.exp(log_beta_q)
    # sample latent_fairness from Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.beta, alpha_q, beta_q)

# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss="ELBO", num_particles=7)

n_steps = 4000
# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

# grab the learned variational parameters
alpha_q = torch.exp(pyro.param("log_alpha_q")).data.numpy()[0]
beta_q = torch.exp(pyro.param("log_beta_q")).data.numpy()[0]

# here we use some facts about the beta distribution
# compute the inferred mean of the coin's fairness

# 2018.4.25 add comment
# beta distribution mean
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * np.sqrt(factor)

print("\nbased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

........................................
based on the data and our prior belief, the fairness of the coin is 0.540 +- 0.090
